In [800]:
import pandas as pd
import numpy as np 

In [801]:
df = pd.read_csv("C:/Users/sebro/Downloads/BD_Supervivencia.csv", encoding="latin-1")
df.head()

,Survived|Pclass|CP_Name|Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|Fare|FROM TO/LUGGAGE-PET
0,B0|3|81912_Mr. Owen Harris Braund|M|22 years|1...
1,e1|_1|32137_Mrs. John Bradley (Florence Briggs...
2,e1|3 _|43466_Miss. Laina Heikkinen|female|26 y...
3,e1|_1|89597_Mrs. Jacques Heath (Lily May Peel)...
4,B0|3|95148_Mr. William Henry Allen|M|35 years|...


In [802]:
# Separamos la columna con el separador | 
cols = [
    'Survived', 'Pclass', 'CP_Name', 'Sex', 'Age',
    'Siblings/Spouses Aboard', 'Parents/Children Aboard',
    'Fare', 'FROM.TO.LUGGAGE.PET'
]

df[cols] = (
    df['Survived|Pclass|CP_Name|Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|Fare|FROM TO/LUGGAGE-PET']
    .str.split('|', expand=True)
)

# Posteriormente eliminamos aquella columna inicial que contenía toda la información de la fila
df1 = df.drop(df.columns[0], axis = 1)

df1.dtypes

Survived                   object
Pclass                     object
CP_Name                    object
Sex                        object
Age                        object
Siblings/Spouses Aboard    object
Parents/Children Aboard    object
Fare                       object
FROM.TO.LUGGAGE.PET        object
dtype: object

In [803]:
# Convirtamos las variables de Parents/Children Aboard y Siblings/Spouses a tipo numérico.
df1['Siblings/Spouses Aboard']  = pd.to_numeric(df1['Siblings/Spouses Aboard'])
df1['Parents/Children Aboard']  = pd.to_numeric(df1['Parents/Children Aboard'])

print('El tipo de dato de Siblings/Spouses Aboard es: ', df1['Parents/Children Aboard'].dtypes, ', y el tipo de dato de Parents/Children Aboard es: ', df1['Parents/Children Aboard'].dtypes)

El tipo de dato de Siblings/Spouses Aboard es:  int64 , y el tipo de dato de Parents/Children Aboard es:  int64


In [804]:
# Eliminar aquellas letras B, e dependiendo de la que toque. Una vez hecho esto, 
# tendremos caracteres en vez de números, entonces después de eliminar estas letras, 
# pasemos nuestros caracteres a números.

df1['Survived'] = df1['Survived'].str.slice(1, 2)
df1['Survived']  = pd.to_numeric(df1['Survived'])
df1[['Survived']].drop_duplicates()

,Survived
0,0
1,1


In [805]:
# Quitaremos sus signos que no nos importan, después de ésto quitaremos espacios para
# que solo nos queden los números para finalmente convertirlo a tipo de dato numérico
df1['Pclass'] = pd.to_numeric(
    df1['Pclass']
    .astype(str)
    .str.extract(r'(\d+)')[0],
    errors='coerce'
)

df1[['Pclass']].drop_duplicates()

,Pclass
0,3
1,1
9,2


In [806]:
# Ahora, para el campo CP_Name lo separaremos en dos, los primeros 5 dígitos en formato numérico
# y el nombre solo va a acontener solo letras, sin acentos y todo en mayúscula.

df1['CP'] = df1['CP_Name'].str.split('_', n=1).str[0]
df1['CP'] = pd.to_numeric(df1['CP'])
df1['Nombre'] = df1['CP_Name'].str.split(' ', n=1).str[1]

# Eliminamos la columna inicial
df1 = df1.drop(df1.columns[2], axis = 1)

df1[['CP','Nombre']].head()

,CP,Nombre
0,81912,Owen Harris Braund
1,32137,John Bradley (Florence Briggs Thayer) Cumings
2,43466,Laina Heikkinen
3,89597,Jacques Heath (Lily May Peel) Futrelle
4,95148,William Henry Allen


In [807]:
# Como pudimos ver en la celda de arriba, tenemos varios registros de nombre en los que hay más nombres entre paréntesis
# para modificar esto, vamos a quedarnos con aquellos nombres que estén entre paréntesis.

parentesis = df1['Nombre'].str.contains(r'\(', na=False)

df1.loc[parentesis, 'Nombre'] = df1.loc[parentesis, 'Nombre'].str.extract(r'\(([^)]+)\)', expand=False)

df1['Nombre']

0            Owen Harris Braund
1        Florence Briggs Thayer
2               Laina Heikkinen
3                 Lily May Peel
4           William Henry Allen
                 ...           
882             Juozas Montvila
883       Margaret Edith Graham
884    Catherine Helen Johnston
885            Karl Howell Behr
886              Patrick Dooley
Name: Nombre, Length: 887, dtype: object

In [808]:
# Ahora, si vemos el campo de Sexo, tendremos lo siguiente
df1['Sex'].unique()
# Vemos que no hay una sola forma de referirnos a los hombres o a las mujeres,
# por lo que para ello, haremos un cambio donde a todos los hombres se les identifique como M y a las mujeres como F

array(['M', 'female', 'F', 'male'], dtype=object)

In [809]:
df1['Sex'] = (
    df1['Sex']
    .replace({
        'female': 'F',
        'male': 'M'
    }))

df1['Sex'].unique()

# Finalmente podemos ver como solo nos quedan una distinción para cada sexo

array(['M', 'F'], dtype=object)

In [810]:
# Finalmente, veamos la variable de edad
df1['Age']
# Arreglaremos el campo de tal forma que solo tendrán datos numéricos de tipo entero.

0          22 years
1          38 years
2          26 years
3          35 years
4          35 years
           ...     
882    27 years old
883    19 years old
884     7 years old
885    26 years old
886    32 years old
Name: Age, Length: 887, dtype: object

In [811]:
df1['Age'] = (
    df1['Age']
    .astype(str)
    .str.replace('years old', '', regex=False)
    .str.replace('years', '', regex=False)
    .str.strip()
)

df1['Age'] = pd.to_numeric(df1['Age'], errors='coerce').astype('int64')
df1['Age'].dtype

dtype('int64')

In [812]:
# Finalmente, tenemos el campo de Fare, donde quitaremos los datos especiales como el de euro
# para que solo queden datos numéricos decimales.
df1['Fare']

0         7.25 £
1      71.2833 £
2        7.925 £
3         53.1 £
4         8.05 £
         ...    
882         13 £
883         30 £
884      23.45 £
885         30 £
886       7.75 £
Name: Fare, Length: 887, dtype: object

In [813]:
df1['Fare'] = (
    np.floor(
        pd.to_numeric(
            df1['Fare'].astype(str).str.replace('£', '').str.strip(),
            errors='coerce'
        )))

df1['Fare']

0       7.0
1      71.0
2       7.0
3      53.0
4       8.0
       ... 
882    13.0
883    30.0
884    23.0
885    30.0
886     7.0
Name: Fare, Length: 887, dtype: float64

In [814]:
# Finalmente, atacaremos la columna de 'FROM.TO.LUGGAGE.PET'
# lo separaremos en 4 columnas nuevas. 
# FROM: primera ciudad de donde parte
# TO: la segunda ciudad, hacia donde se dirige
# LUGGAGE: 1 si tiene equipaje, 0 si no.
# PET: Si si cuenta con equipaje documentado, 0 si no.

tmp = df1['FROM.TO.LUGGAGE.PET'].str.extract(
    r'From\s+(.*?)\s+to\s+(.*?)/(\d+)-(.*)'
)

tmp.columns = ['From', 'To', 'Luggage', 'Pet']
df1 = pd.concat([df1, tmp], axis=1)
df1 = df1.drop(df1.columns[7], axis = 1)

In [815]:
# Finalmente, podemos ver que, nuestros datos iniciales eran:
df.head()

,Survived|Pclass|CP_Name|Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|Fare|FROM TO/LUGGAGE-PET,Survived,Pclass,CP_Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,FROM.TO.LUGGAGE.PET
0,B0|3|81912_Mr. Owen Harris Braund|M|22 years|1...,B0,3,81912_Mr. Owen Harris Braund,M,22 years,1,0,7.25 £,From Southampton to Queenstown/1-no
1,e1|_1|32137_Mrs. John Bradley (Florence Briggs...,e1,_1,32137_Mrs. John Bradley (Florence Briggs Thaye...,female,38 years,1,0,71.2833 £,From Southampton to Queenstown/0-Si
2,e1|3 _|43466_Miss. Laina Heikkinen|female|26 y...,e1,3 _,43466_Miss. Laina Heikkinen,female,26 years,0,0,7.925 £,From Southampton to Cherburgo/0-no
3,e1|_1|89597_Mrs. Jacques Heath (Lily May Peel)...,e1,_1,89597_Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35 years,1,0,53.1 £,From Southampton to Cherburgo/0-Si
4,B0|3|95148_Mr. William Henry Allen|M|35 years|...,B0,3,95148_Mr. William Henry Allen,M,35 years,0,0,8.05 £,From Southampton to Queenstown/0-Si


In [816]:
# Terminando de la siguiente forma
df1.head()

,Survived,Pclass,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,CP,Nombre,From,To,Luggage,Pet
0,0,3,M,22,1,0,7.0,81912,Owen Harris Braund,Southampton,Queenstown,1,no
1,1,1,F,38,1,0,71.0,32137,Florence Briggs Thayer,Southampton,Queenstown,0,Si
2,1,3,F,26,0,0,7.0,43466,Laina Heikkinen,Southampton,Cherburgo,0,no
3,1,1,F,35,1,0,53.0,89597,Lily May Peel,Southampton,Cherburgo,0,Si
4,0,3,M,35,0,0,8.0,95148,William Henry Allen,Southampton,Queenstown,0,Si
